In [2]:
import os
os.chdir('../streamlit')

In [3]:
from utils import load_all_data

all_data = load_all_data(exclude_teg_50 = True, exclude_incomplete_tegs = False)


2025-09-19 16:10:32.697 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-19 16:10:32.701 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-19 16:10:32.702 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-19 16:10:32.703 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-19 16:10:32.704 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-19 16:10:32.705 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-19 16:10:32.707 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-19 16:10:32.708 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-

In [ ]:

# print(all_data.columns)
streaks_data = all_data[['Pl','Sc','Career Count','GrossVP','Hole Order Ever']].copy()

streaks_data['eagle'] = streaks_data['GrossVP']<=-2
streaks_data['birdie'] = streaks_data['GrossVP']<=-1
streaks_data['par_better'] = streaks_data['GrossVP']<=0
streaks_data['double_bogey'] = streaks_data['GrossVP']>1
streaks_data['TBP'] = streaks_data['GrossVP']>2
